
# Klasifikasi LDA
## crawl berita

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrap_berita_detikcom(max_articles=200):
    """
    Mengambil berita dari indeks news.detik.com hingga jumlah artikel maksimal tercapai.
    Kode ini diperbarui untuk menggunakan format URL "?page=" untuk paginasi.
    """
    # URL dasar yang baru sesuai permintaan
    base_url = "https://news.detik.com/indeks"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    data = {
        "id": [],
        "judul_berita": [],
        "kategori_berita": [],
        "isi_berita": []
    }

    id_counter = 1
    page_counter = 1

    # print(f"Memulai scraping berita Detik.com, target: {max_articles} artikel.") # Hapus print

    while id_counter <= max_articles:
        # --- PERUBAHAN UTAMA DI SINI ---
        # Menggunakan format URL dengan parameter "?page="
        url = f"{base_url}?page={page_counter}"
        # print(f"\n--> Mengambil data dari halaman: {page_counter} (Total artikel terkumpul: {id_counter-1})") # Hapus print

        try:
            r = requests.get(url, headers=headers, timeout=15)
            # print(f"    Response Status Code Halaman Indeks: {r.status_code}") # Hapus print
            r.raise_for_status()

            soup = BeautifulSoup(r.content, "html.parser")

            # Selektor untuk menemukan semua blok artikel (masih sama dan valid)
            list_artikel = soup.select("article.list-content__item")

            if not list_artikel:
                # print("    Tidak ada artikel lagi ditemukan di halaman ini. Proses berhenti.") # Hapus print
                break

            # print(f"    Ditemukan {len(list_artikel)} artikel di halaman ini.") # Hapus print

            for artikel in list_artikel:
                if id_counter > max_articles:
                    # print("    Target jumlah artikel tercapai. Menghentikan proses.") # Hapus print
                    break

                link_detail_tag = artikel.select_one("a.media__link")
                if not link_detail_tag or not link_detail_tag.has_attr('href'):
                    continue

                link_detail = link_detail_tag['href']

                if "/video/" in link_detail or "/foto/" in link_detail:
                    continue

                try:
                    response_detail = requests.get(link_detail, headers=headers, timeout=15)
                    response_detail.raise_for_status()
                    soup_detail = BeautifulSoup(response_detail.content, "html.parser")

                    # --- Ekstraksi Data (Selektor masih sama dan valid) ---

                    # Judul Berita: <h1 class="detail__title">
                    judul_tag = soup_detail.select_one("h1.detail__title")
                    judul = judul_tag.get_text(strip=True) if judul_tag else "Judul tidak ditemukan"

                    # Kategori Berita: <div class="page__breadcrumb"> -> <a> terakhir
                    kategori_tags = soup_detail.select("div.page__breadcrumb a")
                    kategori = kategori_tags[-1].get_text(strip=True) if kategori_tags else "Kategori tidak ditemukan"

                    # Isi Berita: <div class="detail__body-text">
                    isi_container = soup_detail.select_one("div.detail__body-text")
                    if isi_container:
                        for s in isi_container.select('script, style, div.lihatjg, table.linksisip'):
                            s.decompose()
                        isi_berita = ' '.join([p.get_text(strip=True) for p in isi_container.find_all("p")])
                    else:
                        isi_berita = "Isi berita tidak ditemukan."

                    if judul != "Judul tidak ditemukan" and isi_berita and isi_berita.strip() != "":
                        data["id"].append(id_counter)
                        data["judul_berita"].append(judul)
                        data["kategori_berita"].append(kategori)
                        data["isi_berita"].append(isi_berita)

                        # print(f"    [+] Berhasil mengambil artikel ID: {id_counter} - Judul: {judul[:50]}...") # Hapus print
                        id_counter += 1

                    time.sleep(0.3)

                except requests.exceptions.RequestException as e_detail:
                    # print(f"    [-] Error saat mengakses detail artikel {link_detail}: {e_detail}") # Hapus print
                    continue
                except Exception as e_process:
                    # print(f"    [-] Error saat memproses artikel {link_detail}: {e_process}") # Hapus print
                    continue

            page_counter += 1
            time.sleep(1)

        except requests.exceptions.RequestException as e_page:
            # print(f"\nTerjadi kesalahan saat mengakses halaman indeks {url}: {e_page}") # Hapus print
            # print("Proses scraping dihentikan.") # Hapus print
            break
        except Exception as e_general:
            # print(f"\nTerjadi kesalahan umum: {e_general}") # Hapus print
            break


    df = pd.DataFrame(data)
    df.to_csv("berita_detikcom_terbaru.csv", index=False)
    # print(f"\n✅ Proses scraping selesai. Total artikel terkumpul: {len(df)}. Data disimpan di 'berita_detikcom_terbaru.csv'") # Hapus print
    return df

# --- Untuk menjalankan scraper ---
if __name__ == '__main__':
    df_hasil = scrap_berita_detikcom(max_articles=200)
    print("\n--- Data Berita Detik.com ---") # Ganti judul
    display(df_hasil) # Gunakan display() untuk tabel pandas


--- Data Berita Detik.com ---


,id,judul_berita,kategori_berita,isi_berita
0,1,Rekrutmen BPJS Kesehatan Khusus Dokter: Syarat...,Berita,BPJS Kesehatanmembuka rekrutmen dan seleksi pe...
1,2,KPK: Nilai Korupsi Kasus Taspen Rp 1 T Bisa Bu...,Berita,KPKmenyebut kasus investasi fiktifPT Taspenyan...
2,3,Gaza Bergemuruh Sambut Pulangnya 2.000 Tahanan...,Foto News,Gaza- Ribuan warga Gaza menyambut gembira pemb...
3,4,Relawan Prabowo-Gibran Akan Bentuk Tim Siber L...,Berita,Sejumlah relawan Presiden Prabowo Subianto dan...
4,5,KPK soal 57 Eks Pegawai Ingin Balik: Kita Horm...,Berita,KPKmerespons soal 57 eks pegawai KPK yang terg...
...,...,...,...,...
195,196,BMKG Prediksi Cuaca Panas Ekstrem Akan Mereda ...,Berita,"Badan Meteorologi, Klimatologi, dan Geofisika ..."
196,197,Bobby Nasution Tawarkan Berbagai Potensi Inves...,Berita,Gubernur Sumatera Utara (Sumut) Muhammad Bobby...
197,198,PSI DKI Dukung Pergub Larang Konsumsi Daging A...,Berita,Anggota Komisi B DPRD DKI Jakarta dari Fraksi ...
198,199,Kementrans Bakal Jemput Bola Kebut Realisasi S...,Berita,Kementerian Transmigrasi RI (Kementrans) optim...


## melakukann praproses untuk data berita yang berhasil didapatkan

In [ ]:
# Cell 1: Import Library
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Library untuk LDA
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel

# Library untuk Klasifikasi dan Evaluasi
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Download resource NLTK (jika belum ada)
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Cell 2: Memuat dan Membersihkan Data Teks
# Muat dataset
df = pd.read_csv('berita_detikcom_terbaru.csv')

# Hapus baris dengan data yang kosong
df.dropna(subset=['isi_berita'], inplace=True)

# Tentukan stopwords bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

def preprocess_text(text):
    # 1. Hapus karakter non-alfabet
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    # 2. Ubah ke huruf kecil
    text = text.lower()
    # 3. Tokenisasi teks menjadi kata-kata
    tokens = word_tokenize(text)
    # 4. Hapus stopwords dan kata-kata pendek (kurang dari 3 huruf)
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    return tokens

# Terapkan fungsi pra-pemrosesan pada kolom 'isi_berita'
df['tokens'] = df['isi_berita'].apply(preprocess_text)

print("Contoh hasil pra-pemrosesan:")
print(df[['isi_berita', 'tokens']].head())

# Menampilkan dataframe
display(df)

Contoh hasil pra-pemrosesan:
                                          isi_berita  \
0  BPJS Kesehatanmembuka rekrutmen dan seleksi pe...   
1  KPKmenyebut kasus investasi fiktifPT Taspenyan...   
2  Gaza- Ribuan warga Gaza menyambut gembira pemb...   
3  Sejumlah relawan Presiden Prabowo Subianto dan...   
4  KPKmerespons soal 57 eks pegawai KPK yang terg...   

                                              tokens  
0  [bpjs, kesehatanmembuka, rekrutmen, seleksi, p...  
1  [kpkmenyebut, investasi, fiktifpt, taspenyang,...  
2  [gaza, ribuan, warga, gaza, menyambut, gembira...  
3  [relawan, presiden, prabowo, subianto, gibran,...  
4  [kpkmerespons, eks, pegawai, kpk, tergabung, i...  


,id,judul_berita,kategori_berita,isi_berita,tokens
0,1,Rekrutmen BPJS Kesehatan Khusus Dokter: Syarat...,Berita,BPJS Kesehatanmembuka rekrutmen dan seleksi pe...,"[bpjs, kesehatanmembuka, rekrutmen, seleksi, p..."
1,2,KPK: Nilai Korupsi Kasus Taspen Rp 1 T Bisa Bu...,Berita,KPKmenyebut kasus investasi fiktifPT Taspenyan...,"[kpkmenyebut, investasi, fiktifpt, taspenyang,..."
2,3,Gaza Bergemuruh Sambut Pulangnya 2.000 Tahanan...,Foto News,Gaza- Ribuan warga Gaza menyambut gembira pemb...,"[gaza, ribuan, warga, gaza, menyambut, gembira..."
3,4,Relawan Prabowo-Gibran Akan Bentuk Tim Siber L...,Berita,Sejumlah relawan Presiden Prabowo Subianto dan...,"[relawan, presiden, prabowo, subianto, gibran,..."
4,5,KPK soal 57 Eks Pegawai Ingin Balik: Kita Horm...,Berita,KPKmerespons soal 57 eks pegawai KPK yang terg...,"[kpkmerespons, eks, pegawai, kpk, tergabung, i..."
...,...,...,...,...,...
195,196,BMKG Prediksi Cuaca Panas Ekstrem Akan Mereda ...,Berita,"Badan Meteorologi, Klimatologi, dan Geofisika ...","[badan, meteorologi, klimatologi, geofisika, b..."
196,197,Bobby Nasution Tawarkan Berbagai Potensi Inves...,Berita,Gubernur Sumatera Utara (Sumut) Muhammad Bobby...,"[gubernur, sumatera, utara, sumut, muhammad, b..."
197,198,PSI DKI Dukung Pergub Larang Konsumsi Daging A...,Berita,Anggota Komisi B DPRD DKI Jakarta dari Fraksi ...,"[anggota, komisi, dprd, dki, jakarta, fraksi, ..."
198,199,Kementrans Bakal Jemput Bola Kebut Realisasi S...,Berita,Kementerian Transmigrasi RI (Kementrans) optim...,"[kementerian, transmigrasi, kementrans, optimi..."


## Membangun Model LDA
Di sini kita akan membuat kamus (dictionary) dan korpus dari teks yang sudah dibersihkan, lalu melatih model LDA. Jumlah topik (num_topics) adalah hyperparameter yang bisa Anda sesuaikan untuk mendapatkan hasil terbaik.

In [ ]:
# Cell 3: Membuat Kamus, Korpus, dan Melatih Model LDA
# Buat kamus dari semua token
dictionary = Dictionary(df['tokens'])

# Filter kata-kata yang terlalu jarang atau terlalu sering muncul
dictionary.filter_extremes(no_below=5, no_above=0.5)

# Buat korpus (Bag-of-Words)
corpus = [dictionary.doc2bow(doc) for doc in df['tokens']]

# Tentukan jumlah topik yang akan diekstraksi
num_topics = 5  # Anda bisa bereksperimen dengan angka ini, misal 5, 10, atau 15

# Latih model LDA
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=100, passes=10, alpha='auto', per_word_topics=True)

# Tampilkan topik-topik yang berhasil diidentifikasi oleh LDA
print("Topik yang berhasil diidentifikasi oleh LDA:")
for idx, topic in lda_model.print_topics(-1):
    print(f"Topik: {idx} \nKata Kunci: {topic}\n")

Topik yang berhasil diidentifikasi oleh LDA:
Topik: 0 
Kata Kunci: 0.025*"kpk" + 0.013*"ketua" + 0.010*"korupsi" + 0.009*"daerah" + 0.008*"panas" + 0.008*"kawasan" + 0.008*"tersangka" + 0.007*"bmkg" + 0.007*"sumatera" + 0.007*"indonesia"

Topik: 1 
Kata Kunci: 0.025*"pesantren" + 0.018*"pemerintah" + 0.016*"program" + 0.014*"sekolah" + 0.012*"sppg" + 0.011*"masyarakat" + 0.010*"presiden" + 0.010*"pendidikan" + 0.009*"cak" + 0.009*"imin"

Topik: 2 
Kata Kunci: 0.036*"korban" + 0.026*"pelaku" + 0.020*"polisi" + 0.017*"sekolah" + 0.015*"anak" + 0.015*"orang" + 0.014*"siswa" + 0.013*"kepala" + 0.012*"motor" + 0.008*"haji"

Topik: 3 
Kata Kunci: 0.023*"prabowo" + 0.019*"presiden" + 0.018*"indonesia" + 0.017*"perdamaian" + 0.014*"menteri" + 0.014*"gaza" + 0.011*"trump" + 0.011*"mesir" + 0.011*"masyarakat" + 0.010*"negara"

Topik: 4 
Kata Kunci: 0.016*"gaza" + 0.016*"jakarta" + 0.014*"israel" + 0.010*"trump" + 0.009*"orang" + 0.008*"hamas" + 0.008*"indonesia" + 0.008*"masyarakat" + 0.008*"pal

## Ekstraksi Fitur dari Dokumen
Setelah model LDA dilatih, kita gunakan model tersebut untuk mengubah setiap berita menjadi sebuah vektor fitur. Vektor ini berisi probabilitas keanggotaan berita tersebut dalam setiap topik.

In [ ]:
# Cell 4: Ekstraksi Fitur Topik untuk Setiap Dokumen
def get_topic_distribution(tokens):
    # Ubah dokumen baru menjadi Bag-of-Words
    bow = dictionary.doc2bow(tokens)
    # Dapatkan distribusi topik dari model LDA
    topic_distribution = lda_model.get_document_topics(bow, minimum_probability=0.0)
    # Buat vektor fitur
    topic_vector = [topic[1] for topic in topic_distribution]
    return topic_vector

# Buat kolom baru yang berisi vektor fitur dari LDA
df['lda_features'] = df['tokens'].apply(get_topic_distribution)

# Tampilkan hasilnya
print("Contoh Vektor Fitur LDA:")
print(df[['judul_berita', 'lda_features']].head())

# Konversi kolom fitur menjadi array NumPy untuk digunakan oleh Scikit-learn
X = np.array(df['lda_features'].tolist())
y = df['kategori_berita']

print("\nDimensi matriks fitur (X):", X.shape)
print("Dimensi target (y):", y.shape)

Contoh Vektor Fitur LDA:
                                        judul_berita  \
0  Rekrutmen BPJS Kesehatan Khusus Dokter: Syarat...   
1  KPK: Nilai Korupsi Kasus Taspen Rp 1 T Bisa Bu...   
2  Gaza Bergemuruh Sambut Pulangnya 2.000 Tahanan...   
3  Relawan Prabowo-Gibran Akan Bentuk Tim Siber L...   
4  KPK soal 57 Eks Pegawai Ingin Balik: Kita Horm...   

                                        lda_features  
0  [0.99153274, 0.0016587629, 0.0024606795, 0.001...  
1  [0.9978498, 0.00042123406, 0.00062487694, 0.00...  
2  [0.0049006687, 0.0040211403, 0.0059651, 0.0047...  
3  [0.00056682015, 0.9975242, 0.00068994414, 0.00...  
4  [0.9974491, 0.0004997317, 0.0007413212, 0.0005...  

Dimensi matriks fitur (X): (200, 5)
Dimensi target (y): (200,)


## Melatih Model Klasifikasi
Sekarang, dengan fitur dari LDA (X) dan label kategori (y), kita dapat melatih model klasifikasi. Di sini kita menggunakan Random Forest, tetapi Anda bisa menggantinya dengan model lain seperti Logistic Regression, SVM, atau Naive Bayes.

In [ ]:
# Identifikasi kategori dengan jumlah data < 2
category_counts = df['kategori_berita'].value_counts()
categories_to_remove = category_counts[category_counts < 2].index.tolist()

print(f"Kategori dengan jumlah data kurang dari 2 yang akan dihapus: {categories_to_remove}")

# Hapus baris yang termasuk dalam kategori yang akan dihapus
df_filtered = df[~df['kategori_berita'].isin(categories_to_remove)].copy()

# Perbarui X dan y dengan data yang sudah difilter
X_filtered = np.array(df_filtered['lda_features'].tolist())
y_filtered = df_filtered['kategori_berita']

print(f"\nJumlah data setelah menghapus kategori minoritas: {len(df_filtered)}")
print(f"Dimensi matriks fitur (X_filtered): {X_filtered.shape}")
print(f"Dimensi target (y_filtered): {y_filtered.shape}")

# Lanjutkan dengan membagi data latih dan data uji menggunakan data yang sudah difilter
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42, stratify=y_filtered)

# Inisialisasi model klasifikasi
classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Latih model dengan data latih
print("\nMelatih model Random Forest Classifier dengan data yang sudah difilter...")
classifier.fit(X_train, y_train)
print("Pelatihan model selesai.")

Kategori dengan jumlah data kurang dari 2 yang akan dihapus: ['Bbc World']

Jumlah data setelah menghapus kategori minoritas: 199
Dimensi matriks fitur (X_filtered): (199, 5)
Dimensi target (y_filtered): (199,)

Melatih model Random Forest Classifier dengan data yang sudah difilter...
Pelatihan model selesai.


##  Evaluasi Performa Model
Terakhir, kita gunakan data uji untuk memprediksi kategori dan membandingkannya dengan kategori sebenarnya untuk mengukur seberapa baik performa model kita.

In [ ]:
# Cell 6: Evaluasi Performa Model
# Lakukan prediksi pada data uji
y_pred = classifier.predict(X_test)

# Evaluasi performa model
print("\nEvaluasi Model Random Forest Classifier:")
print(f"Akurasi: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Evaluasi Model Random Forest Classifier:
Akurasi: 0.8000

Classification Report:
                          precision    recall  f1-score   support

                  Berita       0.82      0.93      0.87        29
                      Dw       0.00      0.00      0.00         1
               Foto News       1.00      1.00      1.00         2
           Internasional       0.50      0.33      0.40         3
Kategori tidak ditemukan       1.00      0.67      0.80         3
                   Kolom       0.00      0.00      0.00         1
  Melindungi Tuah Marwah       0.00      0.00      0.00         1

                accuracy                           0.80        40
               macro avg       0.47      0.42      0.44        40
            weighted avg       0.76      0.80      0.77        40



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
